# <center> Topic Modeling </center>

- This lecture is created based on 
    * Blei. D. (2012), "Probabilistic Topic Models", http://www.cs.columbia.edu/~blei/talks/Blei_ICML_2012.pdf
    * http://videolectures.net/mlss09uk_blei_tm/
    
- An intuitive explanation of LDA: https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d

## 1. Introduction

- Topic modeling provides methods for automatically organizing, understanding, searching, and summarizing large electronic archives.
  - Discover the hidden themes that pervade the collection.
  - Annotate the documents according to those themes.
  - Use annotations to organize, summarize, and search the texts.
  
<img src='topic_modeling.png' width='70%'>

- Formal Definition
  - **Topics**: each topic is a **distribution over words** 
    - e.g. for topic "Gentics", $p('gene'~|~'Genetics')~=~0.04$, $p('dna'~|~'Genetics')=0.02$
    - $K$ topics $\theta_1, \theta_2, ..., \theta_K$, $N$ words $w_1, w_2, ..., w_N$ in corpus, we need to know  $p(w_i|\theta_j)$ for $i \in N$ and $j\in K$
  - **Document ($d$)**: a **mixture of topics**
    - e.g. for above document $d$, $p('Genetics'~|~d)=0.5$, $p('LifeScience'~|~d)=0.15$, ...
    - In general, given document $d$ and topic $\theta_j$, we need to know $p(\theta_j~|~d)$, i.e. **topic proportion**
- Topic models are also referred to as language models



## 2. How to estimate these probabilities?
### 2.1. Review: supervised learning - Naive Bayes
- Topic probability: 
$$ p(\theta_j) = \frac{\text{documents in topic } j +\alpha_1} {\text{total documents}+\alpha_2}$$  
- Word distribution per topic: 
$$ p(w_i~|~\theta_j)= \frac{\text{count of word } w_i \text{ in topic } j + \alpha_1} {\text{total word count in documents of topic }j + \alpha_2}$$  
- Topic distribution per document: 
$$ \begin{array}{l}
 p(\theta_j~|~d) = \frac{p(d|\theta_j) * p(\theta_j)}{p(d)} \text{         # Bayesian rule}\\
 C_{MAP} = \underset{\theta}{\operatorname{argmax}}{p(d~|~\theta)*p(\theta)} \text{         # maximum a posteriori}\\
    C_{MAP} = \underset{\theta}{\operatorname{argmax}}{p(w_1,w_2, ...,w_N~|~\theta)*p(\theta)} \\
    C_{MAP} = \underset{\theta}{\operatorname{argmax}}({\prod_{i \in N} {p(w_i~|~\theta)})*p(\theta)}  \textit{ # independence assumption}
  \end{array}$$  
- Naive Bayes model is also a kind of language model

### 2.2. Generative Model for Unsupervised learning 
- We don't have labeled data; we only observe the documents
- We **cannot** estimate $p(\theta_j)$ and $p(w_i~|~\theta_j)$ as above
- Instead, we use a **generative model** that describes how a document $d$ was created
  1. decide on document length $N$, e.g. 100 words
  2. decide on topic mixture (i.e. $p(\theta_j~|~d)$), e.g. 70% about genetics and 30% about life science, ...
  3. for each of the N words,
     - 3.1. choose a topic from the topic mixture, e.g. "genetics"
     - 3.2. choose a word from based on the probabilities of words in the topic (i.e. $p(w_i~|~\theta_j)$), e.g. "gene"
     - At the end, you may get a document such as "gene dna life ..."
- We assume all documents in the dataset were generated following this process. Then we infer these probabilities from samples such that these probabilities have the maximum likelihood to generate the samples
- Probabilities $p(w_i~|~\theta_j)$ and $p(\theta_j~|~d)$ are **hidden structures** to be discovered, a.k.a **latent variables**

<img src='latent_structure.png' width='70%' style="float: left;">
  


## 3. Latent Dirichlet Allocation (LDA)
### 3.1 Model
- A generative model which generates a document $d$ as follows:
  1. Choose document length $N$ ∼ Poisson(ξ).
  2. Choose topic mixture $\theta$ ~ Dir(α).
  3. For each of the $N$ words $w_n$:
     - (a) Choose a topic assignment $z_n$ ∼ Multinomial(θ), i.e. $p(z_n~|~\theta)$
     - (b) Choose a word $w_n$ from the topic, $w_n$ ∼ Multinomial($\beta_{z_n}$), where $\beta_{z_n}$ is the word distribution for assigned topic $z_n$, i.e. $p(w_n~|~z_n, ~\theta)$     

### 3.2. A few distributions

  - **Poisson**(ξ) : a given number of events occurring in a fixed interval of time/space with rate ξ independently of the time/space since the last event
  - **Multinomial**(θ) & Multinomial($\beta$): 
    - suppose X is a vector which represents n draws of a random variable with three possible outcomes (i.e. words), say A, B, C. 
    - e.g. when n=10, an example draw of X could be x = [4,4,2], i.e., A occured 4 times, B 4 times, and C 2 times 
    - assume three outcomes have probability $\beta$={$\beta_A$, $\beta_B$, $\beta_C$} respectively (i.e. 0.5,0.3,0.2)
    - the multinomial distribution describes the prob. mass distribution of X, $$ P(X=[4,4,2]) = \frac{10!}{4!4!2!}\beta_A^{4}\beta_B^{4}\beta_C^{2}$$ 
  - **Dirichlet** Dir(α) : is a probability distribution with parameter $α, e.g. \{α_1,α_2,α_3\}$ to generate $θ, e.g. \{ θ_1,θ_2,θ_3\}$. For details of Dirichlet function, check videos e.g. https://www.youtube.com/watch?v=nfBNOWv1pgE 
  
    - Dirichlet distribution is conjugate to the multinomial.
    - Given a multinomial observation, the posterior distribution of θ is a Dirichlet.
      - e.g. in the above multinomial example, if $\beta$ ~ $Dir (\alpha) $ (i.e. prior), with samples $X$, $\beta~|~X$ ~ $Dir (\alpha + X)$ (i.e. posterior)
    - In LDA, usually $α_1=α_2=α_3=...=\frac{1}{K}$
    
<img src='dirichlet.svg' width="40%" style="float: center;">

### 3.3. LDA Parameter Fitting
- Common techniques to estimate these probabilities are Variational Bayesian inference, Collapsed Gibbs Sampling (See Blei's paper for details)
  - Collapsed Gibbs Sampling: https://www.youtube.com/watch?v=u7l5hhmdc0M
    1. Randomly assign each word in each sample to a topic 
    2. Iteratively update the assignment of each word

<img src="gibbs_sampling.png" width="70%" style="float: left;">



### 3.4. Evaluate Topic Model - Perplexity
- For a single document $d$ with $N_d$ words, denoted as $\textbf{W}_d = \{w_1, w_2, ..., w_{N_d}\}$,  <br>
$ 
   \begin{align}
  perplexity(d) &= exp({H(d)}),\\
  H(d) &= - \frac{ln (p(\textbf{W}_d))}{N_d}\\
  \end{align}  
$
- $p(\textbf{W}_d)$, the probability of seeing a document $d$, can be calculated based on:
   - word distribution per topic, i.e. $p(w_i~|~\theta_j)$, and 
   - topic mixture, i.e. $p(\theta_j~|~d)$
- For a test set of D with M documents <br>
$ 
   \begin{align}
   perplexity(d) &= exp({H(D)}), \\
   H(D) &= - \frac{\sum_{d \in D} {ln   (p(\textbf{W}_d)})}{\sum_{d \in D}{N_d}} 
   \end{align} 
   $
- Intutition: 
  - A lower perplexity score indicates better generalization performance
  - Minimizing H(d) is equivalent to maximizing log likelihood
- To evaluate a topic model, calcuate perplexity on **testing dataset** (i.e. evaluating how generaalized the model is)
- Note: if you have some labeled data, you should also conduct **external evaluation**, i.e. 
  - map each topic to a labeled class, 
  - compute precision/recall from the labeled data

## 4. Experiement with LDA
- A few libraries available for LDA: gensim, lda, sklearn
- We use sklearn and gensim here 

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Exercise 4.1. Load yahoo multi-label data set
import json
from numpy.random import shuffle

data=json.load(open('../../dataset/ydata.json','r'))

# shuffle the data
shuffle(data)

text,label=zip(*data)
text=list(text)
label=list(label)

print(text[0])
print(label[0])

In [ ]:
# Exercise 4.2. Preprocessing - Create Term Frequency Matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# LDA can only use raw term counts for LDA 
tf_vectorizer = CountVectorizer(max_df=0.90, \
                min_df=50, stop_words='english')
tf = tf_vectorizer.fit_transform(text)

# each feature is a word (bag of words)
# get_feature_names() gives all words
tf_feature_names = tf_vectorizer.get_feature_names()

print(tf_feature_names[0:10])
print(tf.shape)

# split dataset into train (90%) and test sets (10%)
# the test sets will be used to evaluate proplexity of topic modeling
X_train, X_test = train_test_split(\
                tf, test_size=0.1, random_state=0)

In [ ]:
# Exercise 4.3. Train LDA model
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 4

# Run LDA. For details, check
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation.perplexity

# max_iter control the number of iterations 
# evaluate_every determines how often the perplexity is calculated
# n_jobs is the number of parallel threads
lda = LatentDirichletAllocation(n_components=num_topics, \
                                max_iter=20,verbose=1,
                                evaluate_every=1, n_jobs=1,
                                random_state=0).fit(X_train)

In [ ]:
# Exercise 4.4. Check topic and word distribution per topic

num_top_words=20

# lda.components_ returns a KxN matrix
# for word distribution in each topic.
# Each row consists of 
# probability (counts) of each word in the feature space

for topic_idx, topic in enumerate(lda.components_):
    print ("Topic %d:" % (topic_idx))
    # print out top 20 words per topic 
    words=[(tf_feature_names[i],topic[i]) \
           for i in topic.argsort()[::-1][0:num_top_words]]
    print(words)
    print("\n")

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math

num_top_words=50
f, axarr = plt.subplots(2, 2, figsize=(8, 8));

for topic_idx, topic in enumerate(lda.components_):
    # create a dataframe with two columns (word, weight) for each topic
    
    # create a word:count dictionary
    f={tf_feature_names[i]:topic[i] for i in topic.argsort()[::-1][0:num_top_words]}
    
    # generate wordcloud in subplots
    wordcloud = WordCloud(width=480, height=450, margin=0, background_color="black");
    _ = wordcloud.generate_from_frequencies(frequencies=f);
    
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].imshow(wordcloud, interpolation="bilinear");
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].set_title("Topic: "+str(topic_idx));
    _ = axarr[math.floor(topic_idx/2), topic_idx%2].axis('off')

plt.tight_layout()
plt.show()



In [ ]:
# Exercise 4.5. Assign documents to topic
import numpy as np

# Generate topic assignment of each document
topic_assign=lda.transform(X_train)

print(topic_assign[0:5])

# set a probability threshold
# the threshold determines precision/recall
prob_threshold=0.25

topics=np.copy(topic_assign)
topics=np.where(topics>=prob_threshold, 1, 0)
print(topics[0:5])

# How to calulate precision and recall
# if your test data has been labeled

In [ ]:
# Exercise 5.6. Evaluate topic models by perplexity of test data

perplexity=lda.perplexity(X_test)
print(perplexity)


#### 4.1. Find the number of topics ($K$)
- There are no "golden" rules to find K.
- Perplexity may be one way for you to find the number of topics
    - Typically, the best number of topics should be around the **lowest perplexity**
- However, in practice, a few factors need to be considered:
  - It is usually difficult for human to understand or visulaize a big number of topics
  - You may manually scan the data to figure out possible topics in the data, but these topics may not be correlated with the hidden structure discovered by LDA
  - Usually, after LDA, we need manually inspect each discovered topic, merge or trim topics to get semantically coherent but distinguishable topics

In [ ]:
# Exercise 4.1.1 How to find the best number of topics?
# Vary variable num_topics, e.g. set it to 2, 3, 5, ...
# For each value, train LDA model, 
# calculate perplexity on the test data

import numpy as np
import matplotlib.pyplot as plt

result=[]
for num_topics in range(2,15):
    lda = LatentDirichletAllocation(n_components=num_topics, \
                                learning_method='online', \
                                max_iter=10,verbose=0, n_jobs=1,
                                random_state=0).fit(X_train)
    p=lda.perplexity(X_test)
    result.append([num_topics,p])
    print(num_topics, p)



In [ ]:
import pandas as pd
pd.DataFrame(result, columns=["K", "Perlexity"]).plot.line(x='K',y="Perlexity");
plt.show();

## 4.2 Discussion
- With LDA, how to do internal evaluation?
  - cluster separation
  - cluster coherence

## 5. LDA Gensim Package

In [ ]:
# 5.1. Create LDA model using the same TF matrix generated from sklearn

import gensim
from gensim import corpora

# A corpus is TF matrix in the list format, e.g.:
# [[(0, 1), (1,2), (4, 1), ...], [...], ...]
# which shows the first document has words with id=0,1,4
# and the count of word 0 is 1, word 1 is 2, ...

# convert the gensim corpus from the sparse tf matrix
corpus = gensim.matutils.Sparse2Corpus(X_train, \
                            documents_columns=False)

# create the mapping between id and words
id2word={idx:w for idx, w in \
         enumerate(tf_vectorizer.get_feature_names())}

# create a gensim dictionary from the corpus
# a dictionary contains the frequency of each words 
# the mapping between ids and words
dictionary = corpora.Dictionary.from_corpus(corpus, \
                            id2word=id2word)



In [ ]:
# 6.2. Train LDA model

NUM_TOPICS = 4

# for detailed parameters, check
#https://radimrehurek.com/gensim/models/ldamodel.html

ldamodel = gensim.models.\
ldamodel.LdaModel(corpus, alpha='asymmetric',\
                            num_topics = NUM_TOPICS, \
                            id2word=id2word, \
                            iterations=15)

topics = ldamodel.print_topics(num_words=20)
for topic in topics:
    print(topic)

In [ ]:
# 6.3. visualize topics

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [ ]:
# 6.4. Test unseen documents

test_corpus = gensim.matutils.Sparse2Corpus(X_test, \
                    documents_columns=False)
predict = ldamodel.get_document_topics(test_corpus)
list(predict)[0:5]